In [1]:
import warnings
def warn(*args, **kwargs):
    pass
warnings.warn = warn

import qiskit.tools.jupyter
%qiskit_version_table
#%qiskit_copyright

In [2]:
from xbm import get_all_meas_circs_bits_coefs
from utils import eval_expectation
from qiskit import QuantumCircuit, Aer
from qiskit.aqua import QuantumInstance
from qiskit.circuit import ParameterVector
import numpy as np

nb_qubits = 3

p_vec0 = ParameterVector('theta_0', length=2) # qiskit.circuit.parametervector.ParameterVector, parameter vectors for psi_0
p_vec1 = ParameterVector('theta_1', length=2) # qiskit.circuit.parametervector.ParameterVector, parameter vectors for psi_1
p_vec2 = ParameterVector('theta_2', length=2) # qiskit.circuit.parametervector.ParameterVector, parameter vectors for psi_2

psi_0 = QuantumCircuit(nb_qubits) # qiskit.QuantumCircuit
psi_1 = QuantumCircuit(nb_qubits) # qiskit.QuantumCircuit
psi_2 = QuantumCircuit(nb_qubits) # qiskit.QuantumCircuit

psi_0.h(1)
psi_0.rx(p_vec0[0], 1)
psi_0.rz(p_vec0[1], 2)

psi_1.ry(p_vec1[0], 0)
psi_1.rx(p_vec1[1], 2)

psi_2.rx(p_vec2[0], 0)
psi_2.rz(p_vec2[1], 1)
psi_2.h(2)

mat_A  = np.random.random(size=(2**nb_qubits,2**nb_qubits)) + 1j * np.random.random(size=(2**nb_qubits,2**nb_qubits)) # numpy.ndarray
mat_A2 = mat_A @ np.conjugate(mat_A).T

qins_state = QuantumInstance(Aer.get_backend('statevector_simulator')) # qiskit.aqua.QuantumInstance
qins_qasm  = QuantumInstance(Aer.get_backend('qasm_simulator'))        # qiskit.aqua.QuantumInstance

meas_1 = get_all_meas_circs_bits_coefs(psi_0, psi_1, mat_A ) # circuits for <psi_0|A|psi_1>
meas_2 = get_all_meas_circs_bits_coefs(psi_2,  None, mat_A2) # circuits for <psi_2|A^2|psi_2>

# $\langle\psi_0|A|\psi_1\rangle$

In [3]:
np.random.random(42)
params_0 = np.random.random(size=2) # numpy.ndarray, parameter values for psi_0
params_1 = np.random.random(size=2) # numpy.ndarray, parameter values for psi_1

In [4]:
psi_0_gt = qins_state.execute(psi_0.assign_parameters({p_vec0: params_0})).get_statevector()
psi_1_gt = qins_state.execute(psi_1.assign_parameters({p_vec1: params_1})).get_statevector()

In [5]:
exp_gt = np.conjugate(psi_0_gt).T @ mat_A @ psi_1_gt
exp_gt

(0.3235760014648993+1.5772281402478203j)

In [6]:
exp_state, result_state = eval_expectation(meas_1, qins_state, params_dict={p_vec0: params_0, p_vec1: params_1})
exp_qasm , result_qasm  = eval_expectation(meas_1, qins_qasm , params_dict={p_vec0: params_0, p_vec1: params_1})
exp_state, exp_qasm

((0.32357600146489934+1.577228140247821j),
 (0.32367554241333246+1.6148061638238744j))

# $\langle\psi_2|A^2|\psi_2\rangle$

In [7]:
np.random.random(42)
params_2 = np.random.random(size=2) # numpy.ndarray, parameter values for psi_2

In [8]:
psi_2_gt = qins_state.execute(psi_2.assign_parameters({p_vec2: params_2})).get_statevector()

In [9]:
exp_gt = np.conjugate(psi_2_gt).T @ mat_A2 @ psi_2_gt
exp_gt

(6.959812951957787+0j)

In [10]:
exp_state, result_state = eval_expectation(meas_2, qins_state, params_dict={p_vec2: params_2})
exp_qasm , result_qasm  = eval_expectation(meas_2, qins_qasm , params_dict={p_vec2: params_2})
exp_state, exp_qasm

((6.959812951957787+4.791495976028126e-17j), (6.806316907777254+0j))

# $\langle\psi_0|A|\psi_1\rangle + \langle\psi_2|A^2|\psi_2\rangle$

In [11]:
np.random.random(42)
params_0 = np.random.random(size=2)
params_1 = np.random.random(size=2)
params_2 = np.random.random(size=2)

In [12]:
psi_0_gt = qins_state.execute(psi_0.assign_parameters({p_vec0: params_0})).get_statevector()
psi_1_gt = qins_state.execute(psi_1.assign_parameters({p_vec1: params_1})).get_statevector()
psi_2_gt = qins_state.execute(psi_2.assign_parameters({p_vec2: params_2})).get_statevector()

In [13]:
exp_gt = np.conjugate(psi_0_gt).T @ mat_A @ psi_1_gt + np.conjugate(psi_2_gt).T @ mat_A2 @ psi_2_gt
exp_gt

(8.397227633686022+0.7083155833835031j)

In [14]:
exp_state, result_state = eval_expectation(meas_1+meas_2, qins_state, params_dict={p_vec0: params_0, p_vec1: params_1, p_vec2: params_2})
exp_qasm , result_qasm  = eval_expectation(meas_1+meas_2, qins_qasm , params_dict={p_vec0: params_0, p_vec1: params_1, p_vec2: params_2})
exp_state, exp_qasm

((8.39722763368602+0.7083155833835018j),
 (8.331514323263601+0.7142911817063933j))